# Project: NIP
# Subroject: Sylhet Floods, June 2022
## Script: run_modifiedDevries_S1.ipynb
This script runs the modified Devires surface water mapping algorithm on Sentinel-1 data for the Sylet June 2022 flooding. The algorithm was modified and applied by Thomas et al. (2021).

It uses elements created by Jonathan (building off Thomas et al. (2021)) for the time series fusion part of the NIP project, principally:

In [4]:
import ee

In [2]:
ee.Authenticate()

Enter verification code:  4/1AfgeXvsSZP3V-Y89sXDlmpf9wFlNLiXEZK2ji60MJU3AqFqAd_F1_LXYOkM



Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import geemap
from functools import partial
import multiprocessing

In [7]:
# Set the root path for working and saving outputs
rootPath = Path('C:/Users/alexsaunders/Documents/01_uoa/04_git/NIP/Sylhet/DataPreparation')
#list((DevriesPath).iterdir())

In [8]:
# Set the module path for accessing the Devries scripts
module_path = os.path.abspath(os.path.join('../../Fusion/Source'))
if module_path not in sys.path:
    sys.path.append(module_path)
import Devries.DevriesMethods as dv

## PART 0: Load imports

In [9]:
# Load imports:
# (1) FeatureCollectios containing the administrative areas
upazilla = ee.FeatureCollection("projects/ee-nip/assets/ShapeFiles/Admin/Upazilas")
region = ee.FeatureCollection("projects/ee-nip/assets/ShapeFiles/Admin/Regions")
district = ee.FeatureCollection("projects/ee-nip/assets/ShapeFiles/Admin/Districts")
# (2) ImageCollections for the target satellites / sensors 
# Load Sentinel-1 imageCollection
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")

## PART 1: Define the ROI

In [10]:
# Define ROIs for the different admin areas containing Sylhet
roi_upa = upazilla.filter(ee.Filter.eq('NAME_3', 'Sylhet Sadar'))
roi_dist = district.filter(ee.Filter.eq('NAME_2', 'Sylhet'))
roi_reg = region.filter(ee.Filter.eq('NAME_1', 'Sylhet'))

## PART 2: Define the TOI

In [11]:
# Define the time period of interest
start_date = ee.Date('2022-05-01')
end_date = ee.Date('2022-09-01')

## PART 3: Filter the Sentinel-1 collection for ROI and TOI

In [12]:
# Filter the Sentinel-1 collection for ROI and TOI - currently using the Sylhet District
S1Filtered = sentinel1.filterDate(ee.Date(start_date), ee.Date(end_date)).filter(ee.Filter.bounds(roi_dist))

# Check how many images returned, and get the dates and orbit numbers
if S1Filtered == None: 
    print('No images for the collection for the TOI and ROI')
else:
    dates = S1Filtered.aggregate_array("system:time_start").getInfo()
    orbitNumber_start = S1Filtered.aggregate_array("orbitNumber_start").getInfo()
    print(S1Filtered.size().getInfo(),'images for the TOI and ROI')
    print('Image dates: ', pd.to_datetime(dates, unit='ms'))
    print('orbitNumbers: ', orbitNumber_start)

19 images for the TOI and ROI
Image dates:  DatetimeIndex(['2022-05-01 23:47:40', '2022-05-11 11:56:42',
               '2022-05-13 23:47:40', '2022-05-23 11:56:43',
               '2022-05-25 23:47:41', '2022-06-04 11:56:44',
               '2022-06-06 23:47:42', '2022-06-16 11:56:45',
               '2022-06-18 23:47:43', '2022-06-28 11:56:46',
               '2022-06-30 23:47:44', '2022-07-12 23:47:44',
               '2022-07-22 11:56:47', '2022-07-24 23:47:45',
               '2022-08-03 11:56:48', '2022-08-05 23:47:46',
               '2022-08-15 11:56:49', '2022-08-27 11:56:49',
               '2022-08-29 23:47:47'],
              dtype='datetime64[ns]', freq=None)
orbitNumbers:  [43024, 43163, 43199, 43338, 43374, 43513, 43549, 43688, 43724, 43863, 43899, 44074, 44213, 44249, 44388, 44424, 44563, 44738, 44774]


In [13]:
# Get the projection and scale information
proj = S1Filtered.first().select(1).projection().getInfo()
scale = S1Filtered.first().select(1).projection().nominalScale().getInfo()

## PART 4: Chipping
It is necessary to create a grid to chip up the data for the S1 algorithm since geemap export has a maximum file size of around 50 MB

In [14]:
# Create a grid at the appropriate scale and number of pixels (scale = chipsize * resolution)
# Create function to create the grid
def create_grid(roi):
    grid = roi.geometry().coveringGrid(proj=roi.geometry().projection(), scale=1024*scale)
    return grid

In [15]:
# Use the bounding box of the ROI for the grid boundary (can instead use the ROI itself)
roi_bounds = ee.FeatureCollection(roi_dist.first().geometry().bounds())

In [16]:
# Apply the create grid function to create grid as Feature Collection
grid = create_grid(roi_bounds) #roi
grid_size = grid.size().getInfo() # number of chips
grid_list = grid.toList(grid_size)

In [17]:
# Show the grid using geemap
gridCenter = roi_bounds.first().geometry().centroid(1).getInfo()['coordinates']
Map = geemap.Map(center=(gridCenter[1], gridCenter[0]), zoom=9)
Map.addLayer(grid)
Map.addLayer(roi_dist)

Map

Map(center=[24.89262612858011, 92.07147040621301], controls=(WidgetControl(options=['position', 'transparent_b…

## PART 5: Apply the Thomas et al. (2021) modified DeVries surface water algorithm
Run the surface water mapping for all S1 images over the Sylhet flooding and export output geotiffs.

In [18]:
# Import modules from the Devries folder
import importlib

import Devries.ExportChipsHelper as ExportHelper
importlib.reload(ExportHelper)

import Devries.DevriesMethods as dv
importlib.reload(dv)

import Devries.DevriesFloodMap as dvFM
importlib.reload(dvFM)

import Devries.DevriesSingleOrbitFloodMap as dvSOFM
importlib.reload(dvSOFM)

<module 'Devries.DevriesSingleOrbitFloodMap' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Fusion\\Source\\Devries\\DevriesSingleOrbitFloodMap.py'>

In [19]:
# Define the start and end dates for the dry baseline
# VALUES USED IN FIRST VERSION
# baseStart = '2018-10-28'  
# baseEnd = '2018-12-11'

baseStart = '2019-01-10' 
baseEnd = '2019-02-26'

# Provide name for the ROI for naming the exported files
roi_name = 'sylhet_dist'

# Create the output folder for saving the exported files
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/')
outputPath = rootPath/'Sen1MitchellSingleOrbit/dryBaseV2'
outputPath.mkdir(exist_ok=True)

In [21]:
# Loop over each orbit number and chip and apply the surface water mapping algorithm
for i in range(0, len(orbitNumber_start)):
    orbitNumber = orbitNumber_start[i]
    imageDate = dates[i]
    print('Orbit: ', orbitNumber, '. Date: ', pd.to_datetime(imageDate, unit='ms').strftime("%Y%m%d"))
    
    for j in range(0, grid_size):
       
        chip_id = range(grid_size)[j]
        chip = ee.Feature(grid_list.get(chip_id)).geometry()
        
        # Check if tif has already been created before, if so skip
        flood_map_image_path = roi_name + '_' + '{:0>2}'.format(chip_id) + '_' + str(pd.to_datetime(imageDate, unit='ms').strftime("%Y%m%d")) + '_' + str(orbitNumber) + '.tif'
        if os.path.exists(outputPath/flood_map_image_path):
            #print('Already exists')
            continue
        
        print('Chip: ', chip_id)
        
        # Run the flood map image function for the given orbitNumber, date and chip
        flood_map_image = dvSOFM.getGTImageSingleOrbit(chip, baseStart, baseEnd, orbitNumber, imageDate)

        # Export the output flood map image
        #geemap.download_ee_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'])# ERROR "partially initialized module 'rasterio' has no attribute '_loading' (most likely due to a circular import)"
        geemap.ee_export_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'], region=chip, file_per_band=False)

Orbit:  43024 . Date:  20220501
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already exists
Already 

In [17]:
def downloadTile(j, i, grid_size, grid_list, roi_name, imageDate, orbitNumber, outputPath, baseStart, baseEnd, scale, proj, fileList):
    chip_id = range(grid_size)[j]
    chip = ee.Feature(grid_list.get(chip_id)).geometry()
    print('Chip: ', chip_id)

    # Check if tif has already been created before, if so skip
    flood_map_image_path = roi_name + '_' + '{:0>2}'.format(chip_id) + '_' + str(pd.to_datetime(imageDate, unit='ms').strftime("%Y%m%d")) + '_' + str(orbitNumber)
    if flood_map_image_path in fileList:
        print('Already Exists')
        return
    
    flood_map_image_path += '.tif'

    # Run the flood map image function for the given orbitNumber, date and chip
    flood_map_image = dvSOFM.getGTImageSingleOrbit(chip, baseStart, baseEnd, orbitNumber, imageDate)

    # Export the output flood map image
    #geemap.download_ee_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'])# ERROR "partially initialized module 'rasterio' has no attribute '_loading' (most likely due to a circular import)"
    geemap.ee_export_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'], region=chip, file_per_band=False)

In [ ]:
# Loop over each orbit number and chip and apply the surface water mapping algorithm
fileList = ExportHelper.getFileList(outputPath)
for i in range(0, len(orbitNumber_start)):
    orbitNumber = orbitNumber_start[i]
    imageDate = dates[i]
    print('Orbit: ', orbitNumber, '. Date: ', pd.to_datetime(imageDate, unit='ms').strftime("%Y%m%d"))
    if __name__ ==  '__main__': 
        with multiprocessing.Pool(2) as pool:
            pool.map(partial(downloadTile, i, grid_size=grid_size, grid_list=grid_list, roi_name=roi_name, imageDate=imageDate, orbitNumber=orbitNumber, outputPath=outputPath, baseStart=baseStart, baseEnd=baseEnd, scale=scale, proj=proj, fileList=fileList), range(0, grid_size))
    # for j in range(0, grid_size):
       


Orbit:  43024 . Date:  20220501


*** NOT USED - originally tried without chipping but file sizes were too big to export with geemap

In [ ]:
#chip_ids = [item['id'] for item in grid.getInfo()['features']]#['id']
#chip_ids_simple = list(range(grid_size))

In [ ]:
# Get the unique orbit numbers for looping over
orbitNumber_start_unique = np.unique(orbitNumber_start)

# Define the start and end dates for the dry baseline
baseStart = '2018-10-28' 
baseEnd = '2018-12-11'

# Provide name for the ROI for naming the exported files
roi_name = 'sylhet_dist'

# Create the output folder for saving the exported files
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data/Raster/Sylhet/')
outputPath = rootPath/'Sen1MitchellSingleOrbit'
outputPath.mkdir(exist_ok=True)

# Loop over each orbit number and apply the surface water mapping algorithm
for i in range(0, len(orbitNumber_start_unique)):
    orbitNumber = orbitNumber_start_unique[i]
    imageDate = dates[i]
    #print(orbitNumber_start_i)
    
    # Run the flood map image function
    flood_map_image = dvSOFM.getGTImageSingleOrbit(roi_upa.first().geometry(), baseStart, baseEnd, orbitNumber, imageDate)

    # Export the output flood map image
    flood_map_image_path = roi_name + '_' + str(pd.to_datetime(imageDate, unit='ms').strftime("%Y%m%d")) + '_' + str(orbitNumber) + '.tif'
    #geemap.download_ee_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'])# ERROR "partially initialized module 'rasterio' has no attribute '_loading' (most likely due to a circular import)"
    geemap.ee_export_image(flood_map_image, outputPath/flood_map_image_path, scale=scale, crs=proj['crs'], region=roi_upa.first().geometry(), file_per_band=False)
    #An error occurred while downloading.
    #Total request size (51039144 bytes) must be less than or equal to 50331648 bytes.